In [2]:
import cv2
import os
import sys
import numpy as np
from pathlib import Path
import glob
import pandas as pd

In [16]:
t_path = r'E:\\FAX\\4GOD\\SEMESTAR_UNO\\Obrada_slike_i_racunalni_vid\\Projekt\\Dataset\\ImageProcessing\\ImageSegmentation\\'
n_path = r'Dataset/'
data_Path = Path(t_path)

In [17]:
img_Path = list(data_Path.glob(r"*/*.jpg"))
img_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],img_Path))
path_col = pd.Series(img_Path,name="PATH").astype(str)
labels_col = pd.Series(img_Labels,name="CATEGORY")

data = pd.concat([path_col,labels_col],axis=1)
cracked = data[data["CATEGORY"] == "cracked"]
uncracked = data[data["CATEGORY"] == "uncracked"]

cracked.to_csv("TCKcracked.csv")
uncracked.to_csv("TCKuncracked.csv")

In [ ]:
def light(image, bright, contrast):
    bright = bright * 1.2
    contrast = contrast * 2
    image = image * ((contrast/127)+1) - contrast + bright
    image = np.clip(image, 0, 255)
    return np.uint8(image)

In [ ]:
def threshold(image, min_limit=0, max_limit=255, clip=0):
    _, image = cv2.threshold(image, min_limit, max_limit, cv2.THRESH_BINARY_INV)
    return np.uint8(image)

In [ ]:
def canny(image, min_limit = 0, max_limit = 255):
    image = cv2.Canny(image, min_limit, max_limit)
    return np.uint8(image)

In [ ]:
def equalize_light(image, limit=3, grid=(7,7), gray=False):
    if (len(image.shape) == 2):
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        gray = True
    
    clahe = cv2.createCLAHE(clipLimit=limit, tileGridSize=grid)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))

    image = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    if gray: 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return np.uint8(image)

In [ ]:
def image_preprocessor(path):
    image = cv2.imread(path)
    rgbImage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    thresholdImage = threshold(rgbImage, min_limit=150, max_limit=255)
    cannyImage = cv2.Canny(thresholdImage, 90, 100)
    contours,_ = cv2.findContours(cannyImage,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    drawContoursPositive = cv2.drawContours(rgbImage,contours,-1,(255,0,0),1)
    return drawContoursPositive
    

In [ ]:
os.chdir(t_path + f"/{mask}")
for folderName in os.listdir(t_path + '/train/'):
    count = 0
    for image_filename in os.listdir(t_path + '/train/' + folderName): 
        image =  image_preprocessor(t_path + '/train/' + folderName + '/' + image_filename)
        
        path = t_path + f"/{mask}/" + folderName
        doesExist = os.path.exists(path)
        if not doesExist:
            os.makedirs(path)
        os.chdir(t_path + f"/{mask}/" + folderName)
        cv2.imwrite(folderName +'_0'+ str(count) + '.jpg', image)
        count = count + 1
    os.chdir(t_path + f"/{mask}")